In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [2]:
cd ..\experiments\data\human\0.991

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments\data\human\0.991


In [3]:
df=pd.read_csv('trials.csv', sep=',')

In [4]:
import ast
toList = lambda x: ast.literal_eval(x)
df['clicks'] = df['clicks'].apply(toList)

In [5]:
df.sort_values('info_cost')
low_cost = df.loc[df.info_cost == 0.25]
med_cost = df.loc[df.info_cost == 1.0]
high_cost = df.loc[df.info_cost == 4]

In [10]:
count = 0
for seq in df['clicks']:
    if seq == []:
        count += 1
        
count/df['clicks'].size

0.3678385416666667

In [31]:
cost_type = high_cost
brfs_count = 0
dfs_count = 0
top = set([9, 10, 11, 12])
left = set([13, 14, 15, 16])
right = set([5,6,7,8])
bottom = set([1, 2, 3, 4])
for seq in cost_type['clicks']:
    if seq == []:
        continue     
    if len(seq) < 4:
        continue
    first_four = seq[:4]
    if set(first_four) == set([1,5,9,13]):
        if len(seq) > 4:
            rest = seq[4:]
            if len(rest) > 4:
                rest = seq[4:8]
            if set(rest) <= set([2,6,10,14]):
                brfs_count += 1 
        else:
            brfs_count += 1
    elif (set(first_four) == top)or(set(first_four) == right)or(set(first_four) == bottom)or(set(first_four) == left):
        if len(seq) > 4:
            rest = set(seq[4:])
            if len(rest) > 4:
                more = True
                rest = set(seq[4:8])
            if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                if more:
                    rest = set(seq[8:])
                    if len(rest) > 4:
                        rest = set(seq[8:12])
                    if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                            dfs_count += 1                    
                else:
                    dfs_count += 1
        else:
            dfs_count += 1
    else: #between 1 and 3 clicks were made
        if (set(seq) < top)or(set(seq) < right)or(set(seq)< bottom) or (set(seq) < left):
            dfs_count += 1
        elif set(seq) < set([1, 5, 9, 13]):
            brfs_count += 1
        
(dfs_count/cost_type['clicks'].size) * 100

2.4193548387096775

ver 0.991 

values are given in percentages; top to bottom is low, med, and high cost

i stands for initial (only the first 4 clicks) while f stands for full (entire sequence)

| Breadth-First | Depth-First(i)| Depth-First(f)|
| ------------- |:-------------:| -------------:|
| 3.02          | 12.50         |8.47       |
| 6.07          | 12.13         |    9.375      |
| 4.23          | 3.43          |    2.42       |

In [ ]:
def classify(click_seq, num_strategies, strategies):
    prob_array = np.zeros((1, num_strategies))
    for click in click_seq:
        pass
    return prob_array